In [18]:
import requests
import json
import time

def setup_api():
    endpoint = "https://yahoo-finance166.p.rapidapi.com/api/news/list-by-symbol"
    querystring = {"s":"AAPL,GOOGL,TSLA","region":"US","snippetCount":"500"}
    headers = {
        "x-rapidapi-key": "b1eae6d022msh7daf5d27f9b259dp1e9c79jsn92395fddbca6",
        "x-rapidapi-host": "yahoo-finance166.p.rapidapi.com"
    }
    return endpoint, querystring, headers

endpoint, querystring, headers = setup_api()

def make_request():
    response = requests.get(endpoint, headers=headers, params=querystring)
    if response.status_code == 429:
        print("Rate limit hit")
        time.sleep(60)
        return make_request()
    return response

response = make_request()

if response.status_code == 200:
    with open("response_data.json", "w") as file:
        json.dump(response.json(), file, indent=4)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [19]:
def process_data(data):
    articles = []
    if 'data' in data and 'main' in data['data'] and 'stream' in data['data']['main']:
        for item in data['data']['main']['stream']:
            content = item.get('content', {})
            title = content.get('title')
            pub_date = content.get('pubDate')
            provider = content.get('provider', {}).get('displayName')
            url = content.get('clickThroughUrl', {}).get('url') if content.get('clickThroughUrl') else None
            stock_tickers = [ticker.get('symbol') for ticker in content.get('finance', {}).get('stockTickers', [])]
            articles.append({
                'title': title,
                'publication_date': pub_date,
                'provider': provider,
                'url': url,
                'stock_tickers': stock_tickers
            })
    return articles

In [20]:
from rich.console import Console
from rich.table import Table

console = Console()

def display_results():
    response = make_request()
    if response.status_code == 200:
        response_data = response.json()
        processed_data = process_data(response_data)
        console.print("[bold underline cyan]News Articles[/]")
        news_table = Table(title="News Articles", show_header=True, header_style="bold magenta")
        news_table.add_column("Title", overflow="fold")
        news_table.add_column("Publication Date", justify="center")
        news_table.add_column("Provider", justify="center")
        news_table.add_column("URL", overflow="fold")
        news_table.add_column("Stock Tickers", justify="center")
        for article in processed_data:
            news_table.add_row(
                article['title'] or "N/A",
                article['publication_date'] or "N/A",
                article['provider'] or "N/A",
                article['url'] or "N/A",
                ", ".join(article['stock_tickers']) or "N/A"
            )
        console.print(news_table)
    else:
        console.print(f"[bold red]Failed to fetch data. Status code: {response.status_code}[/]")

display_results()

News Articles

                                                   News Articles                                                   
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Title                ┃   Publication Date   ┃       Provider       ┃ URL                  ┃    Stock Tickers    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ After Nvidia         │ 2024-11-24T15:00:35Z │    Yahoo Finance     │ https://finance.yaho │  AVGO, ARM, GOOG,   │
│ earnings, is the AI  │                      │                      │ o.com/news/after-nvi │  CRM, QCOM, META,   │
│ trend still          │                      │                      │ dia-earnings-is-the- │  MSFT, MRVL, NVDA,  │
│ investors’ friend?   │                      │                      │ ai-trend-still-inves │        AMZN         │
│                      │                      │                      │ tors-friend-15003552 │                     │
│                      │                      │                      │ 2.html               │                     │
│ The curious case of  │ 2024-11-24T13:00:00Z │      TechCrunch      │ https://finance.yaho │     GOOG, UBER      │
│ Nebius, the publicly │                      │                      │ o.com/news/curious-c │                     │
│ traded AI            │                      │                      │ ase-nebius-publicly- │                     │
│ infrastructure       │                      │                      │ traded-130000197.htm │                     │
│ 'startup'            │                      │                      │ l                    │                     │
│ Judge to hear        │ 2024-11-25T10:30:57Z │   Associated Press   │ https://finance.yaho │        GOOG         │
│ arguments on whether │                      │       Finance        │ o.com/news/judge-hea │                     │
│ Google's advertising │                      │                      │ r-arguments-whether- │                     │
│ tech constitutes a   │                      │                      │ googles-103057097.ht │                     │
│ monopoly             │                      │                      │ ml                   │                     │
│ Apple’s Cook Joins   │ 2024-11-25T10:21:08Z │      Bloomberg       │ https://finance.yaho │        AAPL         │
│ CEO Summit with      │                      │                      │ o.com/news/apple-chi │                     │
│ China Premier as     │                      │                      │ ef-returns-china-bei │                     │
│ Economy Sags         │                      │                      │ jing-025800892.html  │                     │
│ Meet the Spectacular │ 2024-11-25T09:36:00Z │     Motley Fool      │ https://finance.yaho │  NVDA, AAPL, AMZN,  │
│ Vanguard ETF With    │                      │                      │ o.com/news/meet-spec │     ^GSPC, MSFT     │
│ 45.1% of Its         │                      │                      │ tacular-vanguard-etf │                     │
│ Portfolio Invested   │                      │                      │ -45-093600265.html   │                     │
│ in Nvidia, Apple,    │                      │                      │                      │                     │
│ Microsoft, and       │                      │                      │                      │                     │
│ Amazon               │                      │                      │                      │                     │
│ Apple's Cook makes   │ 2024-11-25T09:30:00Z │ South China Morning  │ https://finance.yaho │        AAPL         │
│ third trip to China  │                      │         Post         │ o.com/news/apples-co │                     │
│ in 2024 amid iPhone  │                      │                      │ ok-makes-third-trip- │                     │
│ slump and AI         │                      │         